In [3]:
import os

root = '/home/aa2650/playground/ECG-FM'
FAIRSEQ_SIGNALS_ROOT = '/home/aa2650/playground/fairseq-signals'
FAIRSEQ_SIGNALS_ROOT = FAIRSEQ_SIGNALS_ROOT.rstrip('/')

In [ ]:
train = 80
validation = 10
test = 10
data_split = f"{train}-{validation}-{test}"

PRETRAINED_MODEL='/home/aa2650/playground/ECG-FM/ckpts/mimic_iv_ecg_physionet_pretrained.pt'
MANIFEST_DIR=f"/home/aa2650/datasets/code_15/manifests/{data_split}"
LABEL_DIR="/home/aa2650/datasets/code_15"
OUTPUT_DIR=f'/home/aa2650/playground/ECG-FM/experiments/{data_split}'
NUM_LABELS=8
# NUM_LABELS=$(($(wc -l < "$/home/aa2650/playground/ECG-FM/data/code_15/labels/label_def.csv") - 1))
# POS_WEIGHT=$(cat $LABEL_DIR/pos_weight.txt)

# checkpoint.save_dir
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [10]:
# build the “finetune” string exactly as before…
finetune_cmd = f"""export HYDRA_FULL_ERROR=1 && \
fairseq-hydra-train \
    task.data={MANIFEST_DIR} \
    model.model_path={PRETRAINED_MODEL} \
    model.num_labels={NUM_LABELS} \
    optimization.lr=[1e-06] \
    optimization.max_epoch=100 \
    dataset.batch_size=128 \
    dataset.num_workers=5 \
    dataset.disable_validation=true \
    distributed_training.distributed_world_size=1 \
    distributed_training.find_unused_parameters=True \
    checkpoint.save_dir={OUTPUT_DIR} \
    checkpoint.save_interval=1 \
    checkpoint.keep_last_epochs=0 \
    common.log_format=csv \
    common.memory_efficient_fp16=True \
    +task.label_file={LABEL_DIR}/y.npy \
    --config-dir {FAIRSEQ_SIGNALS_ROOT}/examples/w2v_cmsc/config/finetuning/ecg_transformer \
    --config-name diagnosis
"""

# wrap with nohup → write both stdout+stderr into train.log
nohup_cmd = f"nohup bash -lc \"{finetune_cmd}\" > {OUTPUT_DIR}/train.log 2>&1 &"

# launch it
os.system(nohup_cmd)
print(f"Launched training under nohup → logs at {OUTPUT_DIR}/train.log")


Launched training under nohup → logs at /home/aa2650/playground/ECG-FM/experiments/80-10-10/train.log


In [6]:
# old command for fine tuning that theoretically should work

# train = 1
# validation = 1
# test = 98
# data_split = f"{train}-{validation}-{test}"

# PRETRAINED_MODEL='/home/aa2650/playground/ECG-FM/ckpts/mimic_iv_ecg_physionet_pretrained.pt'
# MANIFEST_DIR=f"/home/aa2650/datasets/code_15/manifests/{data_split}"
# LABEL_DIR="/home/aa2650/datasets/code_15"
# OUTPUT_DIR=f'/home/aa2650/playground/ECG-FM/experiments/{data_split}'
# NUM_LABELS=8
# # NUM_LABELS=$(($(wc -l < "$/home/aa2650/playground/ECG-FM/data/code_15/labels/label_def.csv") - 1))
# # POS_WEIGHT=$(cat $LABEL_DIR/pos_weight.txt)

# finetune_cmd = f"""fairseq-hydra-train \
#     task.data={MANIFEST_DIR} \
#     model.model_path={PRETRAINED_MODEL} \
#     model.num_labels={NUM_LABELS} \
#     optimization.lr=[1e-06] \
#     optimization.max_epoch=140 \
#     dataset.batch_size=32 \
#     dataset.num_workers=5 \
#     dataset.disable_validation=true \
#     distributed_training.distributed_world_size=1 \
#     distributed_training.find_unused_parameters=True \
#     checkpoint.save_dir={OUTPUT_DIR} \
#     checkpoint.save_interval=1 \
#     checkpoint.keep_last_epochs=0 \
#     common.memory_efficient_fp16=True \
#     common.log_format=csv \
#     +task.label_file={LABEL_DIR}/y.npy \
#     --config-dir {FAIRSEQ_SIGNALS_ROOT}/examples/w2v_cmsc/config/finetuning/ecg_transformer \
#     --config-name diagnosis
# """
# os.system(finetune_cmd)

In [ ]:
# csv_path = "/home/aa2650/datasets/code_15/labels.csv" 
# output_path = "/home/aa2650/datasets/code_15/y.npy"

# df = pd.read_csv(csv_path)

# # Extract only the label columns (drop 'idx')
# label_columns = df.columns[1:]
# labels_df = df[label_columns]

# y_array = labels_df.astype(np.float64).to_numpy()

# np.save(output_path, y_array)

# print(f"Saved y.npy with shape {y_array.shape} and dtype {y_array.dtype}")


In [8]:
# import pandas as pd

# pos_output_path = "/home/aa2650/datasets/code_15/labels_def.csv"

# df = pd.read_csv(csv_path)

# label_cols = df.columns[2:]
# labels_only = df[label_cols]

# pos_count_all = labels_only.sum()
# pos_percent_all = labels_only.mean()

# label_def_new = pd.DataFrame({
#     "name": label_cols,
#     "pos_count_all": pos_count_all.values,
#     "pos_percent_all": pos_percent_all.values
# })

# label_def_new.to_csv(pos_output_path, index=False)
